# Librerias

In [374]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.image import imread
import os

In [375]:
!pip3 install --upgrade wandb

### Sublibrerias de tensorflow -keras-

In [376]:
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from keras import layers
from keras import initializers
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.optimizers import RMSprop, SGD, Adam
from tensorflow.keras import regularizers
from tensorflow.keras.models import load_model
import time

### Abrimos wandb

In [377]:
# import wandb
# from wandb.integration.keras import WandbMetricsLogger, WandbModelCheckpoint
# wandb.login()

### Hiper-parametros

In [378]:
lr = 0.005 #Learning rate
epochs = 25
bs = 120 #batch size
loss = "categorical_crossentropy"

### inicialización de wandb

In [379]:
run = wandb.init(
    settings=wandb.Settings(x_disable_stats=False, x_stats_sampling_interval = 1.0),
    # set the wandb project where this run will be logged
    name = "Prueba de mejor modelo con L1-L2 y dropout",
    project = "Deeplearning_Digit_Recognition",
    group = "BEST WITH REGULATION",

    # track hyperparameters and run metadata with wandb.config
    config = {
        "layer_1": 512,
        #"layer_2": 512,
        "activation_1": "relu",
        #"activation_2": "relu",
        "w_initializer": "RandomNormal",
        "optimizer": "RMSprop",
        "loss": loss,
        "metric": "accuracy",
        "epoch": epochs,
        "batch_size": bs,
        "layer_quantity": 1,
        "regularizer": "L1-L2"
    }
)
time.sleep(3.0)

### Carga de datos

In [380]:
dataset = mnist.load_data()

### Datos de entrenamiento y prueba

In [381]:
(x_train, y_train), (x_test, y_test) = dataset

### Verificamos la forma de los datos

In [382]:
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(60000, 28, 28) (10000, 28, 28) (60000,) (10000,)


### Se aplanan las imagenes y se normalizan

In [383]:
entrada = int(x_train.shape[1]*x_train.shape[2])
stddev = 1/np.sqrt(entrada)
stddev2 = 1/np.sqrt(512)

x_train = x_train.reshape(int(x_train.shape[0]), entrada)
x_test = x_test.reshape(int(x_test.shape[0]), entrada)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train /= 255.
x_test /= 255.

### Se aplica one-hot encoding sobre las categorias

In [384]:
num_classes=10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# Creación de la red neuronal

In [385]:
model_nadol1l2 = keras.Sequential(
    [
        layers.Dense(512, activation = "relu", name = "layer1", 
                     kernel_initializer=initializers.RandomNormal(stddev = stddev),
                     kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-5)),
        Dropuot(0.3)
        # layers.Dense(512, activation = "relu", name = "layer2", 
        #              kernel_initializer=initializers.RandomNormal(stddev = stddev2)),
        
        layers.Dense(num_classes, activation = "softmax", name = "output"),
    ]
) #Arquitectura de la red neuronal

model_nadol1l2.build(input_shape=(None, entrada))
model_nadol1l2.compile(loss = loss, optimizer = RMSprop(learning_rate = lr),metrics = ["accuracy"]) #Función de costo
model_nadol1l2.summary()

SyntaxError: invalid syntax. Perhaps you forgot a comma? (237990274.py, line 6)

### Reinicio de la red neuronal si se necesita

In [ ]:
# for layer in model_nadol1l2.layers:
#    if hasattr(layer, 'kernel_initializer'):
#        new_weights = layer.kernel_initializer(layer.kernel.shape)
#        layer.kernel.assign(new_weights)
#    if hasattr(layer, 'bias_initializer') and layer.bias is not None:
#        new_bias = layer.bias_initializer(layer.bias.shape)
#        layer.bias.assign(new_bias)

### Estructura de la red neuronal

In [ ]:
history = model_nadol1l2.fit(x_train, y_train,
                    batch_size = bs,
                    epochs = epochs,
                    verbose = 1,
                    validation_data = (x_test, y_test),
                    callbacks=[
                        WandbMetricsLogger(log_freq=1),
                        WandbModelCheckpoint("models/model_nadol1l2.keras")
                    ])
run.finish()

In [ ]:
score = model_nadol1l2.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

### Probando el modelo

In [ ]:
mi_modelo = "model_nal1"
path = f"/tf/keras_neural_network/models/{mi_modelo}.keras"
model = load_model(path)

In [ ]:
os.chdir(r"/tf/keras_neural_network/Mis_Tests")

mi_imagen = "Mitres.jpg"
imtest = imread(mi_imagen)
plt.imshow(imtest)
plt.show()

if imtest.ndim == 3:
    imtest = np.dot(imtest[...,:3], [0.2125, 0.7174, 0.0721])
imtest = np.where(imtest < 0.5, 1.0, 0.0)
imtest = imtest.reshape(1, 784).astype("float32")

a = model.predict(imtest)
print(a.shape)
print(a[0])
print(np.where(a[0] == np.amax(a[0]),1.,0.))

In [ ]:
os.chdir(r"/tf/keras_neural_network")
h = 1 #h = 1 : número 2
         #h = 7018 : número 5
         #h = 7024 : número 3

plt.imshow(x_test[h].reshape(28,28))
plt.show()
a=model.predict(x_test)

print(a.shape)
print(a[h])
print(np.where(a[0] == np.amax(a[0]),1.,0.))

print("resultado correcto:")
print(y_test[h])

In [ ]:
#Para guardar el modelo en disco
model_nadol1l2.save("rednuevaactivación.keras")